
# **PROYECTO FINAL CODERHOUSE DATA SCIENCE**


Alumnos 

Gustavo Skidelsky
Juan Simon Astudilla

La NBA es una liga privada de baloncesto profesional que se disputa en Estados Unidos, considerada como la liga de baloncesto más importante del mundo.

Actualmente, la información es uno de los principales instrumentos para las empresas, y gracias a la repercusión mundial de la NBA y a las tecnologías que disponemos, se puede encontrar una gran recopilación de datos que permiten realizar diversos análisis e incluso modelos de predicción.

El objetivo de este estudio consiste en analizar las variables y muestras que componen el dataset elegido, y comprobar diversos modelos que predigan una variable objetivo, en concreto dicha variable es si el equipo local gana o no el partido.

In [34]:
#Importacion de librerias que utilizaremos

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pandas_profiling import ProfileReport
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
import time
from sklearn.metrics import classification_report, confusion_matrix, plot_confusion_matrix, roc_curve, auc, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import plot_roc_curve

## **ANÁLISIS DESCRIPTIVO Y PREPROCESADO**

El dataset seleccionado corresponde a los partidos jugados en la NBA desde 2003.
Se importa el dataframe elegido y se realiza visión global de las variables y muestras para comprenderlo mejor. 

El nombre del dataset es df_games.

In [35]:
#Importación información del dataset df_games
df_games = pd.read_csv('https://raw.githubusercontent.com/gustavoski/coderDataA/main/games_new.csv')
pd.set_option('display.max_columns', None)
print(df_games.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25796 entries, 0 to 25795
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   GAME_DATE_EST     25796 non-null  object 
 1   GAME_ID           25796 non-null  int64  
 2   GAME_STATUS_TEXT  25796 non-null  object 
 3   HOME_TEAM_ID      25796 non-null  int64  
 4   VISITOR_TEAM_ID   25796 non-null  int64  
 5   SEASON            25796 non-null  int64  
 6   TEAM_ID_home      25796 non-null  int64  
 7   PTS_home          25697 non-null  float64
 8   FG_PCT_home       25697 non-null  float64
 9   FT_PCT_home       25697 non-null  float64
 10  FG3_PCT_home      25697 non-null  float64
 11  AST_home          25697 non-null  float64
 12  REB_home          25697 non-null  float64
 13  TEAM_ID_away      25796 non-null  int64  
 14  PTS_away          25697 non-null  float64
 15  FG_PCT_away       25697 non-null  float64
 16  FT_PCT_away       25697 non-null  float6

In [36]:
#Conversion de la variable 'GAME_DATE_EST' a formato fecha
df_games['GAME_DATE_EST'] = pd.to_datetime(df_games['GAME_DATE_EST'])
df_games = df_games.sort_values(by='GAME_DATE_EST', ascending=False)
#Visualización del dataset 'df_games'.
df_games.tail(2)  #Partidos más antiguos

,GAME_DATE_EST,GAME_ID,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,TEAM_ID_away,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
18432,2003-10-06,10300002,Final,1610612763,1610612749,2003,1610612763,105.0,0.494,0.618,0.267,25.0,48.0,1610612749,94.0,0.427,0.7,0.154,20.0,43.0,1
18433,2003-10-05,10300001,Final,1610612762,1610612742,2003,1610612762,90.0,0.457,0.735,0.143,23.0,41.0,1610612742,85.0,0.447,0.5,0.250,20.0,38.0,1


El dataset está compuesto por 20 variables y 25796 muestras, correspondiente a los partidos disputados en la NBA desde 2003 hasta 2022. Prácticamente todas las variables son de tipo float o int, exceptuando dos que son objects. Esto facilitará el análisis descriptivo del dataset y su preprocesado. También se observa que existen algunos valores nulos, los cuales se manipularán antes de los modelos de predicción.

Las columnas o variables son las siguientes:

GAME_DATE_EST: fecha del partido.

GAME_ID: ID del partido.

GAME_STATUS_TEXT: estado del partido (si se detalla 'Final', es que el partido se ha completado).

HOME_TEAM_ID: ID del equipo local.

VISITOR_TEAM_ID: ID del equipo visitante.

SEASON: temporada.

TEAM_ID_home: ID del equipo local (columna duplicada en HOME_TEAM_ID).

PTS_home: cantidad de puntos encestados por el equipo local.

FG_PCT_home: porcentaje efectivo de tiros de campo del equipo local.

FT_PCT_home: porcentaje efectivo de tiros libres del equipo local.

FG3_PCT_home: porcentaje efectivo de triples del equipo local.

AST_home: asistencias del equipo local.

REB_home: rebotes del equipo local.

TEAM_ID_away: ID del equipo visitante (columna duplicada en VISITOR_TEAM_ID).

PTS_away: cantidad de puntos encestados por el equipo visitante.

FG_PCT_away: porcentaje efectivo de tiros de campo del equipo visitante.

FT_PCT_away: porcentaje efectivo de tiros libres del equipo visitante.

FG3_PCT_away: porcentaje efectivo de triples del equipo visitante.

AST_away: asistencias del visitante.

REB_away: rebotes del equipo visitante.

HOME_TEAM_WINS: si el equipo local gana el partido (1), si no lo gana (0). Esta variable será nuestra variable objetivo en los modelos de predicción que se aplicarán.

Cabe destacar que se poseen los ID de los equipos, pero no aparecen sus nombres. Por ello se decide importar un dataset adicional 'teams' de kaggle, con el fin de reemplazar los ID de los equipos por sus nombres y así realizar un análisis descriptivo más completo e interpretar mejor la información.

In [37]:
#Importación e información del dataset 'df_teams'
df_teams = pd.read_csv('https://raw.githubusercontent.com/gustavoski/coderDataA/main/teams.csv')
print(df_teams.info())

#Visualización y tamaño del dataset teams.
print("Tamaño del dataset df_teams:", df_games.shape)
df_teams.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   LEAGUE_ID           30 non-null     int64  
 1   TEAM_ID             30 non-null     int64  
 2   MIN_YEAR            30 non-null     int64  
 3   MAX_YEAR            30 non-null     int64  
 4   ABBREVIATION        30 non-null     object 
 5   NICKNAME            30 non-null     object 
 6   YEARFOUNDED         30 non-null     int64  
 7   CITY                30 non-null     object 
 8   ARENA               30 non-null     object 
 9   ARENACAPACITY       26 non-null     float64
 10  OWNER               30 non-null     object 
 11  GENERALMANAGER      30 non-null     object 
 12  HEADCOACH           30 non-null     object 
 13  DLEAGUEAFFILIATION  30 non-null     object 
dtypes: float64(1), int64(5), object(8)
memory usage: 3.4+ KB
None
Tamaño del dataset df_teams: (25796, 21)


,LEAGUE_ID,TEAM_ID,MIN_YEAR,MAX_YEAR,ABBREVIATION,NICKNAME,YEARFOUNDED,CITY,ARENA,ARENACAPACITY,OWNER,GENERALMANAGER,HEADCOACH,DLEAGUEAFFILIATION
0,0,1610612737,1949,2019,ATL,Hawks,1949,Atlanta,State Farm Arena,18729.0,Tony Ressler,Travis Schlenk,Lloyd Pierce,Erie Bayhawks
1,0,1610612738,1946,2019,BOS,Celtics,1946,Boston,TD Garden,18624.0,Wyc Grousbeck,Danny Ainge,Brad Stevens,Maine Red Claws
2,0,1610612740,2002,2019,NOP,Pelicans,2002,New Orleans,Smoothie King Center,NaN,Tom Benson,Trajan Langdon,Alvin Gentry,No Affiliate


Una vez comprobado el dataset 'df_teams', se reemplazan los valores de la variable 'NICKNAME' en las variables 'HOME_TEAM_ID' y 'VISITOR_TEAM_ID' del dataset 'df_games'.

In [38]:
#Se reemplazan las variables 'HOME_TEAM_ID' y 'VISITOR_TEAM_ID'
df_names = df_teams[['TEAM_ID', 'NICKNAME']]

#Sustitución del ID local por el nickname del equipo
home_names = df_names.copy()
home_names.columns = ['HOME_TEAM_ID', 'NICKNAME']
merge_1 = pd.merge(df_games['HOME_TEAM_ID'], home_names, how ="left", on="HOME_TEAM_ID")  
df_games['HOME_TEAM_ID'] = merge_1['NICKNAME']

#Sustitución del ID visitante por el nickname del equipo
visitor_names = df_names.copy()
visitor_names.columns = ['VISITOR_TEAM_ID', 'NICKNAME']
merge_2 = pd.merge(df_games['VISITOR_TEAM_ID'], visitor_names, how = "left", on="VISITOR_TEAM_ID")
df_games['VISITOR_TEAM_ID'] = merge_2['NICKNAME']

Llama la atención tres variables que pueden presentar información redundante. La primera variable es GAME_STATUS_TEXT, ya que se trata de una variable con valor único. Al solo poseer el valor 'Final' en todas las muestras del dataframe, nos indica que todos los partidos se completaron.

In [39]:
# Contar Final en GAME_STATUS_TEXT
print("Número de veces que aparece el valor 'Final' en la variable GAME_STATUS_TEXT:", df_games['GAME_STATUS_TEXT'].str.contains('Final').sum())

Número de veces que aparece el valor 'Final' en la variable GAME_STATUS_TEXT: 25796


Al existir la misma cantidad de valores 'Final' que de muestras en el dataset, nos asegura que aparece dicho valor en todas ellas.

Las dos siguientes variables detectadas como redundantes son TEAM_ID_home y TEAM_ID_away, ya que son columnas duplicadas de las columnas HOME_TEAM_ID y VISITOR_TEAM_ID, por lo que duplicaria información.

In [40]:
#Eliminación de las columnas mencionadas con anterioridad
df_games = df_games.drop(['GAME_STATUS_TEXT', 'TEAM_ID_home', 'TEAM_ID_away'], axis=1)

### EDA
### ANÁLISIS DESCRIPTIVO
Una vez realizada la primera inspección y limpieza del dataset, se realiza un análisis exploratorio de los datos, en el cual podemos observar la información analítica y estadística de nuestras variables: correlaciones, outliers, etc. Con esta información se podrá realizar el pre-procesado del dataset y manipulación de sus variables.

VARIABLES CON VALORES ÚNICOS

Se comprueba el número de valores únicos que poseen cada variable.

In [41]:
for j in df_games.columns:
    print(j)
    print(df_games[j].value_counts())

Se truncaron las últimas líneas 5000 del resultado de transmisión.
21600939    1
21600948    1
21600947    1
21600946    1
21600952    1
21600949    1
21600950    1
21600951    1
21600954    1
21600953    1
21600955    1
21600956    1
21600957    1
21600958    1
21600959    1
21600964    1
21600963    1
21600962    1
21600961    1
21600960    1
21600965    1
21600991    1
21600993    1
21601042    1
21601015    1
21601026    1
21601020    1
21601021    1
21601022    1
21601027    1
21601024    1
21601025    1
21601023    1
21601030    1
21601029    1
21601028    1
21601031    1
21601033    1
21601034    1
21601035    1
21601032    1
21601037    1
21601038    1
21601039    1
21601036    1
21601041    1
21601014    1
21601016    1
21600994    1
21601017    1
21600996    1
21600997    1
21600998    1
21600995    1
21601003    1
21601002    1
21601001    1
21601000    1
21600999    1
21601007    1
21601004    1
21601005    1
21601006    1
21601008    1
21601013    1
21601010    1
21601012 

Se comprueba por el número de GAME_ID que las muestras que repiten este valor, son filas duplicadas con los mismos valores en todas las variables, exceptuando los equipos que juegan en cada partido.

Por lo que se decide mantener dichas filas, hasta comenzar con los modelos de predicción y así eliminar información redundante.

In [42]:
#Observación de dos filas con el mismo valor en la variable GAME_ID
df_games[df_games['GAME_ID'] == 22000006]

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
2270,2020-12-25,22000006,Timberwolves,Pistons,2020,138.0,0.556,0.692,0.556,25.0,60.0,99.0,0.343,0.84,0.222,14.0,43.0,1
1680,2020-12-25,22000006,Magic,Heat,2020,138.0,0.556,0.692,0.556,25.0,60.0,99.0,0.343,0.84,0.222,14.0,43.0,1


In [43]:
print("Tamaño del dataset df_game:", df_games.shape)

Tamaño del dataset df_game: (25796, 18)


En total existen 29 muestras con la variable GAME_ID duplicada.

MATRIZ DE CORRELACION

La matriz de correlación indica los coeficientes de conexión entre las variables. Cada celda de la tabla muestra el nivel de correlación entre las dos variables.

In [44]:
#Matriz de correlación
plt.figure(figsize=(16, 16))
plt.title('Matriz de correlación')
sns.heatmap(df_games.corr(), square=True, annot=True,cmap="BuPu")

Se observa que las variables que más correlación tienen entre ellas son:

PTS_home, FG_PCT_home, FG3_PCT_home y AST_home.
PTS_away, FG_PCT_away, FG3_PCT_away y AST_away.
Es normal que dichas variables estén correlacionadas entre sí ya que, a más asistencias y porcentaje de triples y tiros de campo se realicen, más puntos se podrán marcar. Las variables FT_PCT_home y FT_PCT_away (tiros libres) no están relacionadas con los puntos que se encestan ya que no son los tiros que más predominan en un partido, y con los que menos puntos se marcan en proporción con triples y tiros de campo, ya que este tipo de tiros solo se realizan con las faltas.

Por lo tanto, las variables 'PTS_home' y 'PTS_away' se eliminarán para los modelos de predicción, ya que si tuvieramos dichos valores, no haría falta predecir si el equipo gana o no. De momento se mantienen para calcular otros datos de interés.

DATOS DESCRIPTIVOS DE INTERES DEL DATASET
- Ranking de victorias de los equipos

In [45]:
#Ranking equipos con más victorias desde 2003 hasta 2019
top_win = np.where(df_games['HOME_TEAM_WINS'] == 1, df_games['HOME_TEAM_ID'], df_games['VISITOR_TEAM_ID'])
top_win = pd.DataFrame(top_win, columns=['TEAM_ID'])
top_win = top_win.value_counts().to_frame().reset_index()
top_win.columns = ['EQUIPO', 'VICTORIAS']
tw = sns.barplot(data=top_win, x='VICTORIAS', y='EQUIPO', palette = 'viridis').set(title='Ranking victorias de los equipos')

Observamos que los Celtics han sido el equipo con más victorias desde que tenemos registros, seguidos de los Heat. En última posición se observan a los Timberwolves y a los Hornet.

- Ranking equipos con más puntos marcados


In [46]:
#Ranking equipos locales con más puntos marcados en casa
teamhome_pts = df_games.groupby(['HOME_TEAM_ID']).agg({'PTS_home':sum}).sort_values(['PTS_home'],ascending=False).reset_index()
teamhome_pts.columns = ['EQUIPO', 'PTS']

#Ranking equipos visitantes con más puntos marcados fuera de casa
teamvisitor_pts = df_games.groupby(['VISITOR_TEAM_ID']).agg({'PTS_away':sum}).sort_values(['PTS_away'],ascending=False).reset_index()
teamvisitor_pts.columns = ['EQUIPO', 'PTS']

#Ranking equipos con más puntos marcados
team_pts = teamhome_pts.merge(teamvisitor_pts[['EQUIPO','PTS']], on='EQUIPO')
team_pts['PTS TOTALES'] = team_pts.sum(axis=1)
team_pts.columns = ['EQUIPO', 'PTS EN CASA', 'PTS FUERA DE CASA', 'PTS TOTALES']
team_pts = team_pts.sort_values(['PTS TOTALES'],ascending=False)
team_pts.head()

,EQUIPO,PTS EN CASA,PTS FUERA DE CASA,PTS TOTALES
2,Spurs,94651.0,91024.0,185675.0
1,Heat,94671.0,90811.0,185482.0
3,Celtics,94337.0,90063.0,184400.0
0,Lakers,96961.0,83934.0,180895.0
5,Rockets,90581.0,87302.0,177883.0


In [47]:
#Gráficas de Ranking equipos con más puntos marcados
sns.factorplot(data=team_pts[0:3], col='EQUIPO', kind="bar", palette = 'BuPu')

Los Spurs son el equipo con más puntos realizados, seguidos de los Heat y los Celtics. Se observa el mismo comportamiento en todas las gráficas, marcandose más puntos en casa que fuera de casa.

- Ranking equipos con más asistencias

In [48]:
#Ranking equipos locales con más asistencias en casa
teamhome_ast = df_games.groupby(['HOME_TEAM_ID']).agg({'AST_home':sum}).sort_values(['AST_home'],ascending=False).reset_index()
teamhome_ast.columns = ['EQUIPO', 'ASISTENCIAS']

#Ranking equipos visitantes con más asistencias fuera de casa
teamvisitor_ast = df_games.groupby(['VISITOR_TEAM_ID']).agg({'AST_away':sum}).sort_values(['AST_away'],ascending=False).reset_index()
teamvisitor_ast.columns = ['EQUIPO', 'ASISTENCIAS']

#Ranking equipos con más asistencias realizadas
team_ast = teamhome_ast.merge(teamvisitor_ast[['EQUIPO','ASISTENCIAS']], on='EQUIPO')
team_ast['ASISTENCIAS TOTALES'] = team_ast.sum(axis=1)
team_ast.columns = ['EQUIPO', 'AST EN CASA', 'AST FUERA DE CASA', 'ASISTENCIAS TOTALES']
team_ast = team_ast.sort_values(['ASISTENCIAS TOTALES'],ascending=False)
team_ast.head()

,EQUIPO,AST EN CASA,AST FUERA DE CASA,ASISTENCIAS TOTALES
2,Spurs,20826.0,19620.0,40446.0
1,Heat,20907.0,19207.0,40114.0
3,Celtics,20793.0,19176.0,39969.0
0,Lakers,21239.0,17955.0,39194.0
5,Rockets,20027.0,18486.0,38513.0


In [49]:
sns.factorplot(data=team_ast[0:3], col="EQUIPO", kind="bar", palette = 'BuPu')

Los Spurs son el equipo con más asistencias realizadas, seguidos de los Heat y los Celtics. Se observa el mismo comportamiento en todas las gráficas, realizandose más asistencias en casa que fuera de casa.

- Ranking equipos con más rebotes


In [50]:
#Ranking equipos locales con más rebotes en casa
teamhome_reb = df_games.groupby(['HOME_TEAM_ID']).agg({'REB_home':sum}).sort_values(['REB_home'],ascending=False).reset_index()
teamhome_reb.columns = ['EQUIPO', 'REBOTES']

#Ranking equipos visitantes con más rebotes fuera de casa
teamvisitor_reb = df_games.groupby(['VISITOR_TEAM_ID']).agg({'REB_away':sum}).sort_values(['REB_away'],ascending=False).reset_index()
teamvisitor_reb.columns = ['EQUIPO', 'REBOTES']

#Ranking equipos con más rebotes
team_reb = teamhome_reb.merge(teamvisitor_reb[['EQUIPO','REBOTES']], on='EQUIPO')
team_reb['REBOTES TOTALES'] = team_ast.sum(axis=1)
team_reb.columns = ['EQUIPO', 'REB EN CASA', 'REB FUERA DE CASA', 'REBOTES TOTALES']
team_reb = team_reb.sort_values(['REBOTES TOTALES'],ascending=False)
team_reb.head()

,EQUIPO,REB EN CASA,REB FUERA DE CASA,REBOTES TOTALES
2,Heat,39709.0,37963.0,80892.0
1,Celtics,39740.0,37670.0,80228.0
3,Spurs,39708.0,38065.0,79938.0
0,Lakers,40554.0,35128.0,78388.0
5,Pistons,38087.0,36232.0,77026.0


In [51]:
sns.factorplot(data=team_reb[0:3], col="EQUIPO", kind="bar", palette = 'BuPu')

Los Heat son el equipo con más rebotes realizados, seguidos de los Celtics y los Spurs. Al igual que anteriormente con los puntos y las asistencias, los rebotes tienen un comportamiento similar, realizandose más rebotes en casa que fuera de casa.

TRANSFORMACIÓN DE VARIABLES CATEGÓRICAS A NUMÉRICAS

Encontramos en el dataset como variable categórica la columna 'GAME_DATE_EST'. Al tratarse de una variable que identifica a las muestras, se decide categorizar la variable en otras tres nuevas variables: 'GAME_YEAR', 'GAME_MONTH' y 'GAME_DAY'.

De esta forma, aunque aumentemos el número de variables, cada variable tendrá menos categorías. No se opta convertir los valores a numéricos con sus frecuencias de aparición ya que los valores están muy distribuidos.

Posteriormente, se borra la variable 'GAME_DATE_EST' para no duplicar información. Se aprovecha para eliminar también la variable 'SEASON' al crearse la reciente variable 'GAME_YEAR'. Al no coincidir en diversos valores, son más exactos los valores de 'GAME_YEAR', ya que en 'SEASON' se muestran años en muestras que concretamente se jugaron al año siguiente.

In [52]:
#Se crean tres nuevas variables a partir de la variable GAME_DATE_EST
df_games['GAME_YEAR'] = pd.to_datetime(df_games['GAME_DATE_EST']).dt.year.apply(str)
df_games['GAME_MONTH'] = pd.to_datetime(df_games['GAME_DATE_EST']).dt.month.apply(str)
df_games['GAME_DAY'] = pd.to_datetime(df_games['GAME_DATE_EST']).dt.day.apply(str)
df_games['GAME_YEAR'] = pd.to_numeric(df_games['GAME_YEAR'])
df_games['GAME_MONTH'] = pd.to_numeric(df_games['GAME_MONTH'])
df_games['GAME_DAY'] = pd.to_numeric(df_games['GAME_DAY'])

#Eliminación de las columnas GAME_DATE_EST
df_games.drop(['GAME_DATE_EST', 'SEASON'], axis=1, inplace=True)



Posteriormente, se convierten las variables 'HOME_TEAM_ID' y 'VISITOR_TEAM_ID' en numéricas por sus frecuencias de aparición, ya que influye el número de veces que haya jugado cada equipo.

In [53]:
#Conversion las variables categóricas a númericas según la frecuencia de aparición
c=['HOME_TEAM_ID','VISITOR_TEAM_ID']
for i in c:
    freq=df_games[i].value_counts().to_dict()
    df_games[i]=df_games[i].map(freq)


TRATAMIENTO DE VALORES NULOS

Por último, antes de comenzar con los modelos de predicción, se realiza una búsqueda y conteo los valores nulos que existen en el dataset.

In [54]:
#Se identifican cuántos valores nulos presenta cada variable
df_games.isnull().sum()

GAME_ID             0
HOME_TEAM_ID        0
VISITOR_TEAM_ID     0
PTS_home           99
FG_PCT_home        99
FT_PCT_home        99
FG3_PCT_home       99
AST_home           99
REB_home           99
PTS_away           99
FG_PCT_away        99
FT_PCT_away        99
FG3_PCT_away       99
AST_away           99
REB_away           99
HOME_TEAM_WINS      0
GAME_YEAR           0
GAME_MONTH          0
GAME_DAY            0
dtype: int64

Se detectan que existen 99 valores nulos en todas la variables que contienen valores nulos. Se muestra un dataset con los registros que solo contienen valores nulos, para así comprenderlos mejor.

In [55]:
#Visualización de las muestras con valores nulos
nan_rows = df_games[df_games.isnull().any(1)]
pd.set_option('max_rows', None)
display(nan_rows)

,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS,GAME_YEAR,GAME_MONTH,GAME_DAY
18324,10300112,828,876,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2003,10,24
18328,10300115,852,908,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2003,10,24
18327,10300114,860,848,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2003,10,24
18326,10300111,855,779,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2003,10,24
18325,10300110,860,854,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2003,10,24
18321,10300108,859,825,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2003,10,24
18323,10300113,922,863,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2003,10,24
18322,10300109,875,891,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2003,10,24
18320,10300116,818,820,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2003,10,24
18329,10300103,915,869,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2003,10,23


In [56]:
print("Tamaño del dataset nan_rows:", nan_rows.shape)

Tamaño del dataset nan_rows: (99, 19)


Se observa que todas las muestras que tienen valores nulos corresponden a partidos jugados en octubre de 2003.

En dichas muestras no se dispone de la información relevante respecto a los partidos y tampoco de si ganaron en casa o no. Se planteó la posibilad de sustituir dichos valores por 0, pero eso no haría más que crear más outliers de cada variable, y serían datos que no corresponderían a la realidad.

También se planteó imputar valores mediante la moda o de forma aleatoria, pero no existe una moda destacable en las variables y de forma aleatoria no tendría sentido que devolviera un valor alto en PTS_home y un valor pequeño en las correlacionadas con esta (con poco porcentaje efectivo de tiros de campo, triples y tiros libres no se pueden marcar puntos).

Se decide eliminar dichas muestras, pues corresponden al 0,38% del dataframe total.

In [57]:
#Eliminación de muestras con valores nulos
df_games = df_games.dropna()

Ahora el dataset está limpio de valores nulos.

Por último se eliminan las variables 'PTS_home' y 'PTS_away' con el fin de aplicar los modelos de predicción, tal y como se ha argumentato en la matriz de correlación, al igual que las filas duplicadas por GAME_ID para eliminar información redundante a la hora de crear los modelos de predicción. Para reducir el números de columnas, se decide utilizar como índice la variable GAME_ID, ahora que es única.

In [58]:
#Eliminación de las variables 'PTS_home' y 'PTS_away'
df_games.drop(['PTS_home', 'PTS_away'], axis=1, inplace=True)

#Eliminación de filas duplicadas
df_games = df_games.drop_duplicates(subset=['GAME_ID'], ignore_index=False)

df_games = df_games.set_index('GAME_ID')

MODELOS DE PREDICCIÓN

En primer lugar se divide el dataset 'df_games' en dos. Por un lado los datos de entrenamiento (X) y por otro el objetivo (Y). Déspues se vuelven a dividir para obtener un conjunto de entrenamiento con el 80% de las muestras (Xtrain e Ytrain) y otro de test con el 20% de las muestras (Xtest e Ytest) para poder comprobar la efectividad de los diferentes modelos.

In [59]:
#División del dataframe en entrenamiento y objetivo
X = df_games.drop(['HOME_TEAM_WINS'], axis=1)
Y = df_games['HOME_TEAM_WINS']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

Previamente se crea una función para generar una matríz de confusión para cada modelo de predicción que visualiza mejor los resultados.

In [60]:
#Función para crear matriz de confusión
def conf_mat_acc(modelo):
    disp = plot_confusion_matrix(modelo, X_train, Y_train,
                               cmap=plt.cm.BuPu, values_format='d')
    true_pred = disp.confusion_matrix[0,0]+disp.confusion_matrix[1,1]
    total_data = np.sum(disp.confusion_matrix)
    accuracy = true_pred/total_data
    plt.show()

REGRESIÓN LOGÍSTICA

Se trata de un modelo y tipo de análisis de regresión utilizado para predecir el resultado de una variable categórica (en nuestro caso si el equipo de casa gana o no) en función de las variables independientes o predictoras.

In [61]:
#Regresión Logística
logreg = LogisticRegression(max_iter=1000)
logreg.fit(X_train, Y_train)

conf_mat_acc(logreg)

Gracias a la regresión logística se consigue un 81% de precisón en dicho modelo.



DECISION TREE

Se trata de un modelo predictivo que divide el espacio de los predictores agrupando observaciones con valores similares para la variable respuesta o dependiente.

In [62]:
dtc = DecisionTreeClassifier()
dtc.fit(X_train, Y_train)
conf_mat_acc(dtc)

Gracias al arbol de decisión se consigue un 81% de precisón en dicho modelo.

K-NEAREST NEIGHBOURS

Se trata de un algoritmo de Machine Learning que pertenece a los algoritmos de aprendizaje supervisado simples y fáciles de aplicar, que pueden ser utilizados para resolver problemas de clasificación y de regresión. El algoritmo clasifica cada dato nuevo en el grupo que corresponda, según tenga k vecinos más cerca de un grupo o de otro.

In [63]:
knn = KNeighborsClassifier(algorithm = 'brute', n_jobs=-1)
knn.fit(X_train, Y_train)
conf_mat_acc(knn)

Gracias a K-Nearest Neighbours se consigue un 68% de precisón en dicho modelo.

NAIVE BAYES

Se trata es un clasificador que aprende de los datos de entrenamiento y luego predice la clase de la instancia de prueba con la mayor probabilidad posterior. También es útil para datos dimensionales altos ya que la probabilidad de cada atributo se estima independientemente.

In [64]:
mnb = MultinomialNB().fit(X_train, Y_train)
conf_mat_acc(mnb)

Gracias a Naive Bayes se consigue un 75% de precisón en dicho modelo.



RANDOM FOREST

Se trata de un estimador que ajusta una serie de clasificadores de árboles de decisión en varias submuestras del conjunto de datos y utiliza el promedio para mejorar la precisión predictiva y controlar el sobreajuste. Se opta por usar 9 arboles decisión como n estimadores y 7 para la profundidad máxima del árbol.


In [65]:
# n_estimators = number of decision trees
rf = RandomForestClassifier(n_estimators=3,  max_depth=7)
rf.fit(X_train, Y_train)
print("score on test: " + str(rf.score(X_test, Y_test)))
print("score on train: "+ str(rf.score(X_train, Y_train)))
conf_mat_acc(rf)

score on test: 0.8019088430074016
score on train: 0.8241453199571442


Gracias a Random Forest se consigue un 80% de precisón en dicho modelo.

CURVA ROC

La curva ROC es una medida de comparación de modelos, que se muestra en una gráfica más adelante para visualizar así los mejores modelos. A mayor valor de AUC, más preciso será el modelo.

In [66]:
# Visualización del Área bajo la curva ROC
disp = plot_roc_curve(logreg, X_test, Y_test)
plot_roc_curve(dtc, X_test, Y_test, ax = disp.ax_)
plot_roc_curve(knn, X_test, Y_test, ax = disp.ax_)
plot_roc_curve(mnb, X_test, Y_test, ax = disp.ax_)
plot_roc_curve(rf, X_test, Y_test, ax = disp.ax_)

Teniendo en cuenta la gráfica de la curva ROC se puede concluir que los mejores modelos son el modelo de Regresión Logística y Naive Bayes. Randon Forest también aparece uno de los mejores modelos al predecir, pero hay que tener en cuenta la pérdida de interpretación, para la regresión no es tan exacto al no poder predecir valores más allá del entrenamiento

Por el contrario, K-Nearest Neighbours se muestra como el peor modelo debido al tiempo de ejecución y su leve precisión en comparación con el resto. EN cuanto al modelo Decision Tree no podemos asegurarnos en su totalidad ya que en el entrenamiento ha dado 1.0, un resultado que es inpensable de conseguir en una predicción

# CONCLUSIÓN

Como conclusión respecto al análisis del dataset, se puede confirmar que dos equipos **no** tienen la misma probabilidad de ganar, aunque se piense que tiene cada uno el 50% de probabilidades, ya que influye hasta si el partido se juega en el estadio local o visitante.
Cabe destacar la aplicación de este tipo de análisis y estudios para la detección de posibles fraudes deportivos, ya sean dentro de los equipos y clubs, o fuera como en casas de apuestas; las cuales fijan sus márgenes de beneficios y cuotas de apuestas en modelos predictivos.